In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from scipy.stats import multivariate_normal

import pandas as pd
import numpy as np

In [11]:
training_set = pd.read_csv('estaturas.xlsx - normales.csv')
valtest_normales = pd.read_csv('estaturas.xlsx - valtest(normales).csv')
valtest_anomalias = pd.read_csv('estaturas.xlsx - valtest(anomalias).csv')
valtest_normales.columns = valtest_anomalias.columns
training_set.columns = valtest_anomalias.columns
training_set['response'] = 0
valtest_anomalias['response'] = 1
valtest_normales['response'] = 0

x_training_set = training_set[['Estatura', 'Edad']]
y_training_set = training_set['response']
x_valtest_normales = valtest_normales[['Estatura', 'Edad']]
y_valtest_normales = valtest_normales['response']
x_valtest_anomalias = valtest_anomalias[['Estatura', 'Edad']]
y_valtest_anomalias = valtest_anomalias['response']


x_cross_validation_vn, x_test_vn, y_cross_validation_vn, y_test_vn = train_test_split(x_valtest_normales, y_valtest_normales, test_size=0.5)
x_cross_validation_va, x_test_va, y_cross_validation_va, y_test_va = train_test_split(x_valtest_anomalias, y_valtest_anomalias, test_size=0.5)


x_cross_validation_set = pd.concat([x_cross_validation_vn, x_cross_validation_va])
y_cross_validation_set = pd.concat([y_cross_validation_vn, y_cross_validation_va])

x_test_set = pd.concat([x_test_vn, x_test_va])
y_test_set = pd.concat([y_test_vn, y_test_va])

In [23]:
def calculate_params(data):
    return np.mean(data, axis=0), np.cov(data.T)

def calculate_function(data, mean, cv):
    return multivariate_normal(mean, cv).pdf(data)

In [26]:
media, cv = calculate_params(x_test_set)
func = calculate_function(x_test_set, media, cv)
func

array([2.02151133e-05, 2.08473714e-05, 1.99386186e-05, 2.12126462e-05,
       2.09535265e-05, 2.05844408e-05, 2.03240350e-05, 2.12245759e-05,
       2.02288438e-05, 1.95292244e-05, 1.98038002e-05, 2.00854743e-05,
       1.99557929e-05, 2.00614063e-05, 2.00854743e-05, 2.12024077e-05,
       2.02030831e-05, 2.00476268e-05, 2.01962020e-05, 2.03429489e-05,
       2.09740765e-05, 2.18733868e-05, 2.00957710e-05, 1.99557929e-05,
       2.16550145e-05, 2.12041149e-05, 2.05655140e-05, 2.04740879e-05,
       2.03309171e-05, 2.07083209e-05, 2.02065219e-05, 1.99695109e-05,
       2.12058219e-05, 2.10867320e-05, 2.29953596e-05, 2.04551850e-05,
       1.99403374e-05, 1.96656026e-05, 1.98038002e-05, 2.21450575e-05,
       2.02065219e-05, 2.08285040e-05, 2.13193379e-05, 2.27028595e-05,
       2.04655004e-05, 2.04637819e-05])

In [ ]:
def predict(data, means, stds, umbral):
    
    def intervals(mean, std, umbral):
        return [mean-(umbral*std), mean+(umbral*std)]
        
    results = []
    for index in range(len(means)):
        intrs = intervals(means[index], stds[index], umbral)
        temp_results = data.apply(lambda x: 0 if (x[0] >= intrs[0] and x[0] <= intrs[1]) else 1, axis=0)
        print(temp_results)
        
def train(x_train_data, y_train_data, x_validate_data, y_validate_data, x_test_data, y_test_data, umbral):
    means, stds = calculate_params(x_train_data)
    
    kf = KFold(n_splits=4)
    folds = kf.get_n_splits(x_validate_data)
    results = []
    for train_index, test_index in kf.split(x_validate_data):
        temp_train_x = x_validate_data.iloc[train_index]
        temp_train_y = np.array(y_validate_data)[train_index]
        
        temp_means, temp_stds = calculate_params(temp_train_x)
        temp_test_x = x_validate_data.iloc[test_index]
        temp_test_y = np.array(y_validate_data)[test_index]
        
        predictions = predict(temp_test_x, temp_means, temp_stds, umbral)
    
train(x_training_set, y_training_set, x_cross_validation_set, y_cross_validation_set, x_test_set, y_test_set, 2)